## Libreria

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import pickle

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

C:\Users\jamr1\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jamr1\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\jamr1\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


## Datos

In [47]:
train_set = pd.read_csv("train_set.csv")
train_set['path']
train_set['array'] = train_set['path'].apply(lambda x: cv2.imread(("train/" + x),cv2.IMREAD_GRAYSCALE))
train_set

,label,id_img,path,array
0,happy,22373,happy/22373.jpg,"[[25, 44, 56, 68, 88, 98, 93, 92, 105, 120, 13..."
1,happy,21433,happy/21433.jpg,"[[33, 29, 22, 18, 19, 23, 22, 19, 20, 23, 14, ..."
2,happy,12418,happy/12418.jpg,"[[35, 43, 66, 84, 71, 41, 36, 53, 97, 56, 100,..."
3,happy,21278,happy/21278.jpg,"[[118, 124, 132, 126, 127, 139, 67, 38, 33, 29..."
4,happy,8081,happy/08081.jpg,"[[201, 209, 193, 107, 106, 107, 98, 114, 144, ..."
...,...,...,...,...
6171,sadness,11346,sadness/11346.jpg,"[[167, 180, 196, 171, 101, 50, 52, 70, 55, 49,..."
6172,sadness,4441,sadness/04441.jpg,"[[163, 154, 128, 116, 121, 106, 90, 99, 115, 1..."
6173,sadness,15236,sadness/15236.jpg,"[[109, 25, 29, 33, 55, 65, 82, 87, 147, 179, 1..."
6174,sadness,27361,sadness/27361.jpg,"[[62, 64, 67, 52, 57, 65, 50, 55, 117, 179, 15..."


In [48]:
df = train_set[['label','array']]
df

,label,array
0,happy,"[[25, 44, 56, 68, 88, 98, 93, 92, 105, 120, 13..."
1,happy,"[[33, 29, 22, 18, 19, 23, 22, 19, 20, 23, 14, ..."
2,happy,"[[35, 43, 66, 84, 71, 41, 36, 53, 97, 56, 100,..."
3,happy,"[[118, 124, 132, 126, 127, 139, 67, 38, 33, 29..."
4,happy,"[[201, 209, 193, 107, 106, 107, 98, 114, 144, ..."
...,...,...
6171,sadness,"[[167, 180, 196, 171, 101, 50, 52, 70, 55, 49,..."
6172,sadness,"[[163, 154, 128, 116, 121, 106, 90, 99, 115, 1..."
6173,sadness,"[[109, 25, 29, 33, 55, 65, 82, 87, 147, 179, 1..."
6174,sadness,"[[62, 64, 67, 52, 57, 65, 50, 55, 117, 179, 15..."


## Tratamiento de Datos

In [49]:
df['target'] = df['label'].map({'happy': 0, 'sadness': 1})


C:\Users\jamr1\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [50]:
data = df[['array', 'target']]

In [51]:
data

,array,target
0,"[[25, 44, 56, 68, 88, 98, 93, 92, 105, 120, 13...",0
1,"[[33, 29, 22, 18, 19, 23, 22, 19, 20, 23, 14, ...",0
2,"[[35, 43, 66, 84, 71, 41, 36, 53, 97, 56, 100,...",0
3,"[[118, 124, 132, 126, 127, 139, 67, 38, 33, 29...",0
4,"[[201, 209, 193, 107, 106, 107, 98, 114, 144, ...",0
...,...,...
6171,"[[167, 180, 196, 171, 101, 50, 52, 70, 55, 49,...",1
6172,"[[163, 154, 128, 116, 121, 106, 90, 99, 115, 1...",1
6173,"[[109, 25, 29, 33, 55, 65, 82, 87, 147, 179, 1...",1
6174,"[[62, 64, 67, 52, 57, 65, 50, 55, 117, 179, 15...",1


## Split Datos

In [52]:
def pixels_to_array(pixels):
    array = np.array(pixels,'float64')
    return array

In [53]:
def image_reshape(data):
    image = np.reshape(data['pixels'].to_list(),(data.shape[0],48,48,1))
    return image

In [54]:
data['pixels'] = data['array'].apply(pixels_to_array)
X = image_reshape(data)
y = data['target']

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1)

In [11]:
X_train = X_train / 255.
X_valid = X_valid / 255.

In [12]:
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

(4940, 48, 48, 1)
(4940,)
(1236, 48, 48, 1)
(1236,)


## Modelo

In [13]:
cnn = Sequential([
    Conv2D(64, (3,3), activation = 'relu', padding = 'same', input_shape=(48,48,1)),
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.25),
    BatchNormalization(),

    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.5),
    BatchNormalization(),
    
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.5),
    BatchNormalization(),

    Flatten(),
    
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.25),
    BatchNormalization(),
    Dense(7, activation='softmax')
])

cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 64)        0         
                                                                 
 batch_normalization (BatchN  (None, 24, 24, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 128)       7

In [14]:
opt = tf.keras.optimizers.Adam(0.001)

cnn.compile(loss='sparse_categorical_crossentropy', 
            optimizer=opt, metrics=['accuracy'])

## Fit

In [15]:
h1 = cnn.fit(
    X_train, y_train, 
    batch_size=32,
    epochs = 20,
    validation_data = (X_valid, y_valid)
)

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
155/155 [==============================] - ETA: 0s - loss: 1.8279 - accuracy: 0.3597WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x000001D26429EEE8> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`

In [16]:
h2 = cnn.fit(
    X_train, y_train, 
    batch_size=32,
    epochs = 20,
    verbose = 1,
    validation_data = (X_valid, y_valid))

Epoch 1/20
155/155 [==============================] - 4s 28ms/step - loss: 0.2779 - accuracy: 0.8868 - val_loss: 0.3286 - val_accuracy: 0.8665
Epoch 2/20
155/155 [==============================] - 4s 28ms/step - loss: 0.2855 - accuracy: 0.8858 - val_loss: 0.3894 - val_accuracy: 0.8220
Epoch 3/20
155/155 [==============================] - 4s 28ms/step - loss: 0.2788 - accuracy: 0.8850 - val_loss: 0.3568 - val_accuracy: 0.8374
Epoch 4/20
155/155 [==============================] - 4s 28ms/step - loss: 0.2721 - accuracy: 0.8848 - val_loss: 0.3177 - val_accuracy: 0.8730
Epoch 5/20
155/155 [==============================] - 4s 28ms/step - loss: 0.2546 - accuracy: 0.8988 - val_loss: 0.3568 - val_accuracy: 0.8487
Epoch 6/20
155/155 [==============================] - 4s 28ms/step - loss: 0.2312 - accuracy: 0.9075 - val_loss: 0.3220 - val_accuracy: 0.8754
Epoch 7/20
155/155 [==============================] - 4s 28ms/step - loss: 0.2360 - accuracy: 0.9057 - val_loss: 0.4707 - val_accuracy: 0.7791

In [18]:
print(h2.params)
print(h2.epoch)
h2.history

{'verbose': 1, 'epochs': 20, 'steps': 155}
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


{'loss': [0.27789974212646484,
  0.28547146916389465,
  0.27882248163223267,
  0.27210402488708496,
  0.25460362434387207,
  0.2311777025461197,
  0.2359994500875473,
  0.2381913959980011,
  0.2199874073266983,
  0.21987831592559814,
  0.20849132537841797,
  0.19917142391204834,
  0.18346336483955383,
  0.2206346094608307,
  0.19861657917499542,
  0.2073187381029129,
  0.1776432991027832,
  0.17440496385097504,
  0.16654086112976074,
  0.1674613356590271],
 'accuracy': [0.8868421316146851,
  0.8858299851417542,
  0.8850202560424805,
  0.8848178386688232,
  0.898785412311554,
  0.9074898958206177,
  0.9056680202484131,
  0.9046558737754822,
  0.9111335873603821,
  0.9117408990859985,
  0.9149797558784485,
  0.9216599464416504,
  0.9251012206077576,
  0.912145733833313,
  0.9206477999687195,
  0.9170040488243103,
  0.928744912147522,
  0.9315789341926575,
  0.9354251027107239,
  0.9374493956565857],
 'val_loss': [0.32857146859169006,
  0.38943079113960266,
  0.3568047285079956,
  0.31773

In [19]:
pd.DataFrame(h2.history)

,loss,accuracy,val_loss,val_accuracy
0,0.277900,0.886842,0.328571,0.866505
1,0.285471,0.885830,0.389431,0.822006
2,0.278822,0.885020,0.356805,0.837379
3,0.272104,0.884818,0.317738,0.872977
4,0.254604,0.898785,0.356753,0.848706
5,0.231178,0.907490,0.322024,0.875405
6,0.235999,0.905668,0.470730,0.779126
7,0.238191,0.904656,0.334944,0.872168
8,0.219987,0.911134,0.471311,0.813916
9,0.219878,0.911741,0.319837,0.869741


## Result

In [20]:
results = cnn.evaluate(X_valid, y_valid)
results

39/39 [==============================] - 0s 9ms/step - loss: 0.3524 - accuracy: 0.8778


[0.35244855284690857, 0.8778316974639893]

## Test

In [44]:
test = pd.read_csv("test_set.csv")
test['array'] = test['path'].apply(lambda x: cv2.imread((x),cv2.IMREAD_GRAYSCALE))
test

,id_img,path,array
0,18341,test/18341.jpg,"[[124, 69, 19, 5, 8, 9, 18, 34, 23, 4, 0, 2, 7..."
1,13176,test/13176.jpg,"[[7, 3, 2, 5, 5, 2, 2, 5, 46, 35, 37, 37, 19, ..."
2,23945,test/23945.jpg,"[[88, 78, 136, 145, 128, 75, 203, 130, 178, 14..."
3,15968,test/15968.jpg,"[[254, 255, 255, 255, 254, 254, 254, 255, 255,..."
4,18382,test/18382.jpg,"[[9, 2, 8, 25, 12, 55, 143, 140, 89, 70, 56, 6..."
...,...,...,...
4112,8966,test/08966.jpg,"[[168, 168, 168, 169, 169, 169, 170, 170, 162,..."
4113,12111,test/12111.jpg,"[[11, 14, 15, 12, 9, 10, 12, 14, 9, 9, 9, 9, 1..."
4114,16629,test/16629.jpg,"[[131, 89, 70, 158, 246, 81, 70, 86, 87, 83, 8..."
4115,24322,test/24322.jpg,"[[20, 27, 28, 23, 29, 45, 56, 55, 46, 57, 108,..."


In [45]:
test['pixels'] = test['array'].apply(pixels_to_array)
Xt = image_reshape(test)

In [56]:
Xt_ = Xt/255
Xt_

array([[[[0.48627451],
         [0.27058824],
         [0.0745098 ],
         ...,
         [0.29803922],
         [0.35686275],
         [0.33333333]],

        [[0.4       ],
         [0.22745098],
         [0.0745098 ],
         ...,
         [0.22352941],
         [0.29019608],
         [0.27843137]],

        [[0.22745098],
         [0.12941176],
         [0.03921569],
         ...,
         [0.15294118],
         [0.22352941],
         [0.22352941]],

        ...,

        [[0.17647059],
         [0.2       ],
         [0.24313725],
         ...,
         [0.5372549 ],
         [0.26666667],
         [0.24705882]],

        [[0.29411765],
         [0.27058824],
         [0.23921569],
         ...,
         [0.76078431],
         [0.43921569],
         [0.2627451 ]],

        [[0.76078431],
         [0.67843137],
         [0.54509804],
         ...,
         [0.88235294],
         [0.58823529],
         [0.30588235]]],


       [[[0.02745098],
         [0.01176471],
         [0.00

In [57]:
Xt_.shape

(4117, 48, 48, 1)

In [76]:
sample1 = cnn.predict(Xt_).argmax(axis=1)

In [79]:
sample1

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [80]:
label = []
for i in sample1:
    if i==0:
        label.append('happy')
    else:
        label.append('sadness')
label

['happy',
 'happy',
 'sadness',
 'happy',
 'happy',
 'happy',
 'sadness',
 'sadness',
 'sadness',
 'happy',
 'sadness',
 'happy',
 'happy',
 'happy',
 'happy',
 'happy',
 'happy',
 'sadness',
 'sadness',
 'sadness',
 'happy',
 'happy',
 'happy',
 'sadness',
 'happy',
 'happy',
 'sadness',
 'happy',
 'sadness',
 'sadness',
 'happy',
 'happy',
 'sadness',
 'happy',
 'happy',
 'sadness',
 'sadness',
 'sadness',
 'happy',
 'sadness',
 'sadness',
 'sadness',
 'happy',
 'happy',
 'happy',
 'sadness',
 'happy',
 'sadness',
 'happy',
 'happy',
 'sadness',
 'happy',
 'happy',
 'happy',
 'happy',
 'happy',
 'sadness',
 'happy',
 'happy',
 'sadness',
 'happy',
 'sadness',
 'sadness',
 'happy',
 'sadness',
 'happy',
 'happy',
 'happy',
 'sadness',
 'happy',
 'sadness',
 'happy',
 'sadness',
 'happy',
 'happy',
 'happy',
 'happy',
 'sadness',
 'sadness',
 'happy',
 'sadness',
 'happy',
 'sadness',
 'sadness',
 'happy',
 'happy',
 'happy',
 'happy',
 'happy',
 'happy',
 'sadness',
 'sadness',
 'sadn

## Submission

In [90]:
sample_sub = pd.read_csv("sample_submission.csv")

In [91]:
sample_sub['label.'] = label

In [92]:
sample_sub = sample_sub.drop('label',axis=1)

In [93]:
sample_sub['label'] = sample_sub['label.']

In [95]:
sample_sub = sample_sub.drop('label.', axis=1)
sample_sub

,id_img,label
0,18341,happy
1,13176,happy
2,23945,sadness
3,15968,happy
4,18382,happy
...,...,...
4112,8966,happy
4113,12111,sadness
4114,16629,happy
4115,24322,happy


In [98]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample_sub.shape:
        if df_to_submit.columns.all() == sample_sub.columns.all():
            if df_to_submit.id_img.all() == sample_sub.id_img.all():
                print("You're ready to submit!")
                sample_sub.to_csv("submission.csv", index = False) #muy importante el index = False
                # urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                # img = Image.open("gfg.png")
                # img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [99]:
chequeator(sample_sub)

You're ready to submit!
